# Problem: Implement a CNN for CIFAR-10 in PyTorch

### Problem Statement
You are tasked with implementing a **Convolutional Neural Network (CNN)** for image classification on the **CIFAR-10** dataset using PyTorch. The model should contain convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification. Your goal is to complete the CNN model by defining the necessary layers and implementing the forward pass.

### Requirements
1. **Define the CNN Model**:
   - Add **convolutional layers** for feature extraction.
   - Add **pooling layers** to reduce the spatial dimensions.
   - Add **fully connected layers** to output class predictions.
   - The model should be capable of processing input images of size `(32x32x3)` as in the CIFAR-10 dataset.

### Constraints
- The CNN should be designed with multiple convolutional and pooling layers followed by fully connected layers.
- Ensure the model is compatible with the CIFAR-10 dataset, which contains 10 classes.


<details>
  <summary>💡 Hint</summary>
  Add the convolutional (conv1, conv2), pooling (pool), and fully connected layers (fc1, fc2) in CNNModel.__init__.
  <br>
  Implement the forward pass to process inputs through these layers.
</details>

In [12]:
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import torch

In [13]:
# Load CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the CNN Model
class CNNModel(nn.Module):
    """A CNN model."""

    def __init__(self):
        """Initializes a CNNModel instance."""
        super().__init__()

        # Here is a tip: At the beginning, write the dimensions. Therefore, you'll run into lesser dimension errors.
        # You can also multiply individual channel sizes :)
        self.conv1 = nn.Conv2d(3, 8, 3, padding=1)  # (3, 32, 32) -> (8, 32, 32)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)  # (8, 32, 32) -> (16, 32, 32)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)  # (16, 32, 32) -> (32, 32, 32)
        self.pool1 = nn.MaxPool2d(2, stride=2)  # (32, 32, 32) -> (32, 16, 16)
        self.flatten = nn.Flatten()  # (32, 16, 16) -> (8192, )
        self.fc1 = nn.Linear((32 * 16 * 16), 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

        self.relu = nn.ReLU()

    def forward(self, x):
        """Forward pass.

        Args:
            x: Input value.
        """
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pool1(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))

        return self.fc3(x)

In [18]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 0.6757
Epoch [2/10], Loss: 0.8307
Epoch [3/10], Loss: 0.5798
Epoch [4/10], Loss: 0.7335
Epoch [5/10], Loss: 0.2545
Epoch [6/10], Loss: 0.0069
Epoch [7/10], Loss: 0.2876
Epoch [8/10], Loss: 0.0170
Epoch [9/10], Loss: 0.1689
Epoch [10/10], Loss: 0.4468


In [21]:
# Evaluate on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 66.23%
